In [1]:
import numpy as np
from scipy.optimize import curve_fit

In [30]:
def slit_to_bars(slit):
    return (slit*2-1, slit*2)

def barpos_to_pix(pos, bar01xcenter, bar01ycenter, bar_width, scale, bar_angle, slit_angle):
    barposmatrix = np.array([ [scale*np.cos(bar_angle),
                               -np.sin(bar_angle)],
                              [np.sin(bar_angle),
                               scale*np.cos(bar_angle)] ])
    slit, mm = pos
    print(slit, mm)
    print(bar01xcenter)
    print(bar01ycenter)
    barpos = np.array([bar01xcenter - mm, slit*bar_width + bar01ycenter])
    xy = np.dot(barpos, barposmatrix)
    return xy


In [31]:
xdata = np.array([ (721, 2022), # pixels
          (1068, 1934),
          (984, 1804),
          (1112, 40),
        ])
ydata = np.array([ (2, 179.155), # slit number, mm
          (6, 133.901),
          (12, 144.962),
          (92, 129.033)
        ])
print(xdata.shape)
print(xdata[0])

(4, 2)
[ 721 2022]


In [ ]:
p0=[269., 255., -5.48, 1./0.124, -0.22*np.pi/180., (4.0-0.22)*np.pi/180.]
result = curve_fit(barpos_to_pix, xdata, ydata, p0=p0)
result